Aqui, vamos usar um conjunto de dados sobre consumo de energia e vamos entender alguns tópicos:

-  indexação baseada em tempo
-  visualização de dados de séries temporal
-  sazonalidade
-  frequências
-  reamostragem
-  Rolling Windows
-  tendências

Assim como em toda análise, o intuito é obter informações relevantes a partir dos dados. Para tanto, é preciso fazer as corretas perguntas que irão direcionar toda a jornada da análise. No nosso caso, temos essas perguntas:

1) Quando o consumo de eletricidade é tipicamente alto e tipicamente baixo?

2) Como a produção de energia, seja solar ou eólica, varia de acordo com as estações do ano?

3) Quais são as tendências de longo prazo em consumo de energia, produção solar e produção eólica?

4) Como a produção eólica e solar são comparadas com o consumo de energia e como essa taxa varia em relação ao tempo?

## Leitura e breve análise dos dados

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Display figures inline in Jupyter notebook
#%matplotlib inline

# Use seaborn style defaults and set the default figure size
sns.set(rc={'figure.figsize':(11, 4)})

In [ ]:
opsd_daily = pd.read_csv('opsd_germany_daily.csv')
opsd_daily.shape

In [ ]:
opsd_daily.head()

In [ ]:
opsd_daily.tail(3) #tail mostra as n últimas linhas

In [ ]:
opsd_daily.dtypes

podemos ver que a coluna Date é um objeto, indicando que o Pandas a trata como texto. Precisamos convertê-la para valores date/time.

In [ ]:
opsd_daily['Date'] = pd.to_datetime(opsd_daily['Date'])
opsd_daily.dtypes

In [ ]:
opsd_daily = opsd_daily.set_index('Date')
opsd_daily.head(3)

In [ ]:
opsd_daily['Year'] = opsd_daily.index.year
opsd_daily['Month'] = opsd_daily.index.month
opsd_daily['Weekday Name'] = opsd_daily.index.weekday_name

In [ ]:
#Mostrando 5 linhas aleatórias
opsd_daily.sample(5, random_state=0) #random_state = seed

## Indexação baseada em tempo

In [ ]:
# selecionando dados referente a um único dia
opsd_daily.loc['2017-08-10']

In [ ]:
# selecionando dados referente a um bloco de dias
opsd_daily.loc['2014-01-20':'2014-01-22']

In [ ]:
# selecionando dados usando string parcial como indexação
#opsd_daily.loc['2006'] seleciona todo o ano de 2006
opsd_daily.loc['2012-02'] # seleciona o mês de fevereiro de 2012

## Visualizando dados de séries temporais

In [ ]:
opsd_daily['Consumption'].plot(linewidth=0.5);

In [ ]:
cols_plot = ['Consumption', 'Solar', 'Wind']
axes = opsd_daily[cols_plot].plot(marker='.', alpha=0.5, linestyle='None', figsize=(11, 9), subplots=True)
for ax in axes:
    ax.set_ylabel('Daily Totals (GWh)')

Podemos tirar algumas conclusões:

    > o consumo de energia é maior no inverno (aquecedor e iluminação) e menor no verão
    > é possível observar dois agrupamentos em relação ao consumo: um em azul mais escuro e o outro em azul mais claro. Vamos investigar isso melhor depois
    > a produção de energia solar é maior no verão e menor no inverno
    > a produção de energia eólica é maior no inverno e menor no verão
    > há uma forte tendência de crescimento na produção de energia eólica ao longo dos anos

In [ ]:
ax = opsd_daily.loc['2017', 'Consumption'].plot()
ax.set_ylabel('Daily Consumption (GWh)');

In [ ]:
ax = opsd_daily.loc['2017-01':'2017-02', 'Consumption'].plot(marker='o', linestyle='-')
ax.set_ylabel('Daily Consumption (GWh)');

## Sazonalidade

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(11, 10), sharex=True)
for name, ax in zip(['Consumption', 'Solar', 'Wind'], axes):
    sns.boxplot(data=opsd_daily, x='Month', y=name, ax=ax)
    ax.set_ylabel('GWh')
    ax.set_title(name)

    # Remove the automatic x-axis label from all but the bottom subplot
    if ax != axes[-1]:
        ax.set_xlabel('')

In [ ]:
sns.boxplot(data=opsd_daily, x='Weekday Name', y='Consumption');

## Frequências

In [ ]:
pd.date_range('1998-03-10', '1998-03-15', freq='D')

Algumas opções de frequenências disponíveis no Pandas são:

    - D: diário
    - H: de hora em hora
    - B: dia de trabalho
    - W: ssemanalmente
    - M: Mensalmente
    - Q: Quinzenalmente
    - Y: Anualmente

In [ ]:
# aqui definimos periods=8, que indica a divisão da freq em periodos sequenciais
pd.date_range('2004-09-20', periods=8, freq='H')

In [ ]:
opsd_daily.index

In [ ]:
times_sample = pd.to_datetime(['2013-02-03', '2013-02-06', '2013-02-08'])

consum_sample = opsd_daily.loc[times_sample, ['Consumption']].copy()
consum_sample

In [ ]:
consum_freq = consum_sample.asfreq('D')

consum_freq['Consumption - Forward Fill'] = consum_sample.asfreq('D', method='ffill')
consum_freq

## Reamostragem

In [ ]:
# reamostragem monstrando a média semanal
data_columns = ['Consumption', 'Wind', 'Solar', 'Wind+Solar']

opsd_weekly_mean = opsd_daily[data_columns].resample('W').mean()
opsd_weekly_mean.head(3)

In [ ]:
start, end = '2017-01', '2017-06'

fig, ax = plt.subplots()

ax.plot(opsd_daily.loc[start:end, 'Solar'],
        marker='.', linestyle='-', linewidth=0.5, label='Daily')

ax.plot(opsd_weekly_mean.loc[start:end, 'Solar'],
        marker='o', markersize=8, linestyle='-', label='Weekly Mean Resample')

ax.set_ylabel('Solar Production (GWh)')
ax.legend();

## Rolling Windows

In [ ]:
opsd_7d = opsd_daily[data_columns].rolling(7, center=True).mean()
opsd_7d.head(10)

In [ ]:
start, end = '2017-01', '2017-06'

fig, ax = plt.subplots()

ax.plot(opsd_daily.loc[start:end, 'Solar'],
        marker='.', linestyle='-', linewidth=0.5, label='Daily')

ax.plot(opsd_weekly_mean.loc[start:end, 'Solar'],
        marker='o', markersize=8, linestyle='-', label='Weekly Mean Resample')

ax.plot(opsd_7d.loc[start:end, 'Solar'],
        marker='.', linestyle='-', label='7-d Rolling Mean')

ax.set_ylabel('Solar Production (GWh)')
ax.legend();

## Tendência

In [ ]:
opsd_365d = opsd_daily[data_columns].rolling(window=365, center=True, min_periods=360).mean()

In [ ]:
# plot diário, média movel de 7 dias e média movel de 365 dias
import matplotlib.dates as mdates # visuaalizar linhas de grid verticais
fig, ax = plt.subplots()

ax.plot(opsd_daily['Consumption'], marker='.', markersize=2, color='0.6',
        linestyle='None', label='Daily')

ax.plot(opsd_7d['Consumption'], linewidth=2, label='7-d Rolling Mean')

ax.plot(opsd_365d['Consumption'], color='0.2', linewidth=3,
        label='Trend (365-d Rolling Mean)')

ax.xaxis.set_major_locator(mdates.YearLocator())
ax.legend()
ax.set_xlabel('Year')
ax.set_ylabel('Consumption (GWh)')
ax.set_title('Trends in Electricity Consumption');

In [ ]:
# Plot média móvel de 365 dias de produção de energia solar e eólica
fig, ax = plt.subplots()
for nm in ['Wind', 'Solar', 'Wind+Solar']:
    ax.plot(opsd_365d[nm], label=nm)

ax.xaxis.set_major_locator(mdates.YearLocator())
ax.set_ylim(0, 400)
ax.legend()
ax.set_ylabel('Production (GWh)')
ax.set_title('Trends in Electricity Production (365-d Rolling Means)');

## Leituras adicionais:
    - https://jakevdp.github.io/PythonDataScienceHandbook/03.11-working-with-time-series.html
    - https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html
    - https://www.kaggle.com/thebrownviking20/everything-you-can-do-with-a-time-series